In [24]:
import numpy as np
from PIL import Image
import scipy as sp 
import os
from pathlib import Path
from tqdm import tqdm 
import re
import skimage as ski

In [2]:
source_files = """
Y:/kau/диплом/data/brodatz-textures/1.1.01.tiff
Y:/kau/диплом/data/brodatz-textures/1.1.02.tiff
Y:/kau/диплом/data/brodatz-textures/1.1.03.tiff
Y:/kau/диплом/data/brodatz-textures/1.1.04.tiff
Y:/kau/диплом/data/brodatz-textures/1.1.05.tiff
Y:/kau/диплом/data/brodatz-textures/1.1.06.tiff
Y:/kau/диплом/data/brodatz-textures/1.1.07.tiff
Y:/kau/диплом/data/brodatz-textures/1.1.08.tiff
Y:/kau/диплом/data/brodatz-textures/1.1.09.tiff
Y:/kau/диплом/data/brodatz-textures/1.1.10.tiff
Y:/kau/диплом/data/brodatz-textures/1.1.11.tiff
Y:/kau/диплом/data/brodatz-textures/1.1.12.tiff
Y:/kau/диплом/data/brodatz-textures/1.1.13.tiff
Y:/kau/диплом/data/brodatz-textures/1.2.01.tiff
Y:/kau/диплом/data/brodatz-textures/1.2.02.tiff
Y:/kau/диплом/data/brodatz-textures/1.2.03.tiff
Y:/kau/диплом/data/brodatz-textures/1.2.04.tiff
Y:/kau/диплом/data/brodatz-textures/1.2.05.tiff
Y:/kau/диплом/data/brodatz-textures/1.2.06.tiff
Y:/kau/диплом/data/brodatz-textures/1.2.07.tiff
Y:/kau/диплом/data/brodatz-textures/1.2.08.tiff
Y:/kau/диплом/data/brodatz-textures/1.2.09.tiff
Y:/kau/диплом/data/brodatz-textures/1.2.10.tiff
Y:/kau/диплом/data/brodatz-textures/1.2.11.tiff
Y:/kau/диплом/data/brodatz-textures/1.2.12.tiff
Y:/kau/диплом/data/brodatz-textures/1.2.13.tiff
Y:/kau/диплом/data/brodatz-textures/1.3.01.tiff
Y:/kau/диплом/data/brodatz-textures/1.3.02.tiff
Y:/kau/диплом/data/brodatz-textures/1.3.03.tiff
Y:/kau/диплом/data/brodatz-textures/1.3.04.tiff
Y:/kau/диплом/data/brodatz-textures/1.3.05.tiff
Y:/kau/диплом/data/brodatz-textures/1.3.06.tiff
Y:/kau/диплом/data/brodatz-textures/1.3.07.tiff
Y:/kau/диплом/data/brodatz-textures/1.3.08.tiff
Y:/kau/диплом/data/brodatz-textures/1.3.09.tiff
Y:/kau/диплом/data/brodatz-textures/1.3.10.tiff
Y:/kau/диплом/data/brodatz-textures/1.3.11.tiff
Y:/kau/диплом/data/brodatz-textures/1.3.12.tiff
Y:/kau/диплом/data/brodatz-textures/1.3.13.tiff
""".split()

len(source_files), source_files[:3]

(39,
 ['Y:/kau/диплом/data/brodatz-textures/1.1.01.tiff',
  'Y:/kau/диплом/data/brodatz-textures/1.1.02.tiff',
  'Y:/kau/диплом/data/brodatz-textures/1.1.03.tiff'])

In [3]:
def split_image_in_chunks(image, chunk_size):
    n1 = image.shape[0] // chunk_size
    n2 = image.shape[1] // chunk_size
    chunks = []
    for i in range(n1):
        for j in range(n2):
            chunks.append(image[i*chunk_size:(i+1)*chunk_size, j*chunk_size:(j+1)*chunk_size])
    return chunks

In [9]:
img = Image.open(source_files[0])
imgd = np.asarray(img)
imgd.shape, imgd.dtype

((512, 512), dtype('uint8'))

In [40]:
lbp = ski.feature.local_binary_pattern(imgd, 1, 8, 'default')
lbp.shape, lbp.dtype

((512, 512), dtype('float64'))

In [16]:
list(map(int,re.match(r'(\d+).(\d+).(\d+)', "1.2.13").groups()))

[1, 2, 13]

In [ ]:
# For Brodatz Textures
CHUNK_SIZE = 100
dataset = []
labels = []
for path in tqdm(source_files, desc="images"):
    path = Path(path)
    img = Image.open(path)
    stem, suffix = path.stem, path.suffix
    
    _, subset, label = map(int, re.match(r'(\d+).(\d+).(\d+)', stem).groups())

    chunks = split_image_in_chunks(np.asarray(img), CHUNK_SIZE)
    dataset += chunks
    labels += [(subset, label)] * len(chunks)

    img.close()

images: 100%|██████████| 39/39 [00:00<00:00, 87.54it/s]


In [ ]:
np.save("Y:/kau/диплом/data/dataset/chunks.npy", np.array(dataset), dtype=np.uint8)
np.save("Y:/kau/диплом/data/dataset/labels.npy", np.array(labels), dtype=np.uint8)

---

In [43]:
a = np.array([1.0, 2.0, 10.0, 130.0])
a.astype(np.uint16)

array([  1,   2,  10, 130], dtype=uint16)

In [69]:
def LBP(img, R, P, method):
    assert method != 'var'
    if method == 'default':
        assert P <= 16
    lbp = ski.feature.local_binary_pattern(img, P, R, method)
    return lbp.astype(np.uint16)

In [53]:
dataset = "Y:/kau/диплом/data/dataset"
# chunks.npy
# labels.npy

labels = np.load(f"{dataset}/labels.npy")
labels.shape, labels.dtype

((1950, 2), dtype('uint8'))

In [63]:
np.shape(labels.flat)

(3900,)

In [70]:
# calculate lbp for whole image and then divide to avoid chunk boundary issues
def lbp_for_chunks(R, P, method):
    codename = f"lbp_R{R}_P{P}_{method[0]}"
    filename = f"{dataset}/{codename}.npy"
    print(f"{codename}..", end='')

    if Path(filename).is_file():
        print(f"exists, skipping.")
        return
    lbped_chunks = []
    for path in source_files:
        path = Path(path)
        img = Image.open(path)
        data = np.asarray(img)
        assert data.dtype == np.uint8
        assert len(data.shape) == 2
        lbp = LBP(data, R, P, method)
        lbped_chunks += split_image_in_chunks(lbp, CHUNK_SIZE)
        img.close()
    print('saving..', end='')
    assert len(lbped_chunks) == labels.shape[0]
    np.save(filename, np.array(lbped_chunks))
    print('done!')


In [72]:
%%time
lbp_for_chunks(1,8,'default')
lbp_for_chunks(2,12,'default')
lbp_for_chunks(1,8,'uniform')
lbp_for_chunks(2,12,'uniform')
lbp_for_chunks(3,24,'uniform')
lbp_for_chunks(5,36,'uniform')

lbp_R1_P8_d..saving..done!
lbp_R2_P12_d..saving..done!
lbp_R1_P8_u..saving..done!
lbp_R2_P12_u..saving..done!
lbp_R3_P24_u..saving..done!
lbp_R5_P36_u..saving..done!
CPU times: total: 48.2 s
Wall time: 49.4 s


---

In [64]:
def lbphist_for_chunks(R, P, method):
    codename = f"R{R}_P{P}_{method[0]}"
    lbp_filename = f"{dataset}/lbp_{codename}.npy"
    hist_filename = f"{dataset}/hist_{codename}.npy"
    print(f"{codename}..", end='')

    if not Path(lbp_filename).is_file():
        print("lbp absent.")
        return
    if Path(hist_filename).is_file():
        print("hist present, skipping.")
        return

    if method == 'uniform':
        nbins = P+2
    elif method == 'default':
        nbins = 2**P
    else:
        print(f"unsupported method '{method}'.")
        return
    
    lbpdata = np.load(lbp_filename)
    assert lbpdata.shape[0] == labels.shape[0]
    assert max(lbpdata.shape[1], lbpdata.shape[2]) < 2**16

    lbphist = np.zeros((labels.shape[0], nbins), dtype=np.uint32)
    for n,chunk in enumerate(lbpdata):
        lbphist[n] = np.bincount(chunk.flat, minlength=nbins)

    assert lbphist[0].sum() == lbpdata.shape[1] * lbpdata.shape[2]
    np.save(hist_filename, lbphist)
    print("done!")

In [73]:
%%time
lbphist_for_chunks(1,8,'default')
lbphist_for_chunks(2,12,'default')
lbphist_for_chunks(1,8,'uniform')
lbphist_for_chunks(2,12,'uniform')
lbphist_for_chunks(3,24,'uniform')
lbphist_for_chunks(5,36,'uniform')

R1_P8_d..done!
R2_P12_d..done!
R1_P8_u..done!
R2_P12_u..done!
R3_P24_u..done!
R5_P36_u..done!
CPU times: total: 641 ms
Wall time: 759 ms


---

In [74]:
a = np.load(f'{dataset}/hist_R1_P8_d.npy')
a

array([[ 829,  132,   24, ...,   34,   98,  820],
       [ 783,  162,   27, ...,   43,  131,  793],
       [ 790,  112,   30, ...,   30,  103,  799],
       ...,
       [ 960,  168,   47, ...,   59,  153, 1091],
       [ 868,  130,   69, ...,   67,  130,  963],
       [ 920,  151,   76, ...,   73,  133, 1047]],
      shape=(1950, 256), dtype=uint32)

In [75]:
b = np.load(f'{dataset}/lbp_R1_P8_d.npy')
b

array([[[ 65,   1, 129, ..., 225,   0,  16],
        [192, 255, 239, ..., 144, 246, 254],
        [193, 224, 243, ...,   7,   0, 240],
        ...,
        [  0, 255,  12, ...,  64, 248, 255],
        [  4, 251, 196, ...,  64, 248, 250],
        [  4, 251, 225, ...,   0, 184, 240]],

       [[ 17,   1,   1, ..., 177,  64, 241],
        [255, 175,   6, ...,  63,   0, 208],
        [112,  55,   4, ..., 255, 135,   0],
        ...,
        [134, 254, 255, ...,  57,   1,  64],
        [255, 193,   0, ...,  31, 159, 132],
        [243,   0,  20, ..., 191, 207, 133]],

       [[  0, 176, 241, ..., 241, 225, 225],
        [252, 255, 227, ...,   1,   0,  16],
        [145,  64, 112, ..., 135, 207,   6],
        ...,
        [127, 143,   6, ..., 108, 124, 255],
        [248, 255, 239, ..., 240, 120, 251],
        [128, 240, 247, ...,  64, 120, 251]],

       ...,

       [[  0, 223, 131, ..., 224, 112, 112],
        [215, 195, 135, ...,  32,  56, 249],
        [  3, 131, 199, ...,  28, 255,   1